In [1]:
import argparse
import json
import logging
import os
import random
import time

import torch
import numpy as np


In [2]:
import sys 
sys.path.append('..')
import utils.misc as ws
import utils.data_utils
import utils.train_utils
import utils.eval_utils
import utils.mesh
import utils.dataset as d
import models.networks as arch


In [26]:
DATA_SOURCE = '/home/ninad/Desktop/Docs/phd-res/proj-irvl-grasp-transfer/code/docker-data/output_dataset/'
# EXPERIMENTS_DIR = '../experiments/epochs_2000_weight_0.5/'
EXPERIMENTS_DIR = '../experiments/all3_dsdf'
CHECKPOINT = 'latest'
LATENT_CODE_DIR = ws.latent_codes_subdir

specs_filename = os.path.join(EXPERIMENTS_DIR, "specs.json")

In [28]:
object_model = '003_cracker_box_google_16k_textured_scale_1000'
path_to_norm_npz = os.path.join(DATA_SOURCE, object_model, "norm_params.npz")
data = np.load(path_to_norm_npz)

obj_scale = data['scale']
obj_offset = data['offset']

print(obj_scale)
print(obj_offset)

0.419322772641168
[-0.012431   -0.01178597  0.10826332]


In [15]:
specs = json.load(open(specs_filename))
latent_size = specs["CodeLength"]
gripper_weight = specs["GripperWeight"]

In [16]:
decoder = arch.dsdfDecoder(latent_size, **specs["NetworkSpecs"])

decoder = torch.nn.DataParallel(decoder)

saved_model_state = torch.load(
    os.path.join(
        EXPERIMENTS_DIR, ws.model_params_subdir, CHECKPOINT + ".pth")
)

saved_model_epoch = saved_model_state["epoch"]

decoder.load_state_dict(saved_model_state["model_state_dict"])

decoder.eval()

decoder = decoder.module.cuda()

In [17]:
# Load the latent vectors learned during the training 
latent_vecs = ws.load_latent_vectors(EXPERIMENTS_DIR, CHECKPOINT)
print(latent_vecs.shape)

torch.Size([120, 256])


In [18]:
# For object point cloud sdf inference, any one latent vector is fine. 
# Later we can store the best scoring vector as a part of the model state.
l_vec = latent_vecs[0]
l_vec = l_vec.squeeze()
print(l_vec.shape)

torch.Size([256])


In [24]:
# Load the query points here
# Right now just doing a random sample
queries = utils.eval_utils.sample_uniform_points_in_unit_sphere(amount=50000)

# Perform the normalization here: Using the scale and offset
queries = (queries - obj_offset) / obj_scale

In [ ]:
# Inference
# The input queries are also a part of the output (ignore this)
# The second var in the returned tuple is the object sdfs
# The third var are gripper sdfs (ignore this)
with torch.no_grad():
    # queries, sdf_obj, sdf_grp = utils.eval_utils.eval_random_query_pc(decoder, l_vec.cuda(), )
    queries, sdf_obj, _ = utils.eval_utils.eval_query_pc(decoder, l_vec, queries, max_batch_size=40000)